In [ ]:
import os
import time
import joblib
import h5py
import random as python_random

import tensorflow as tf
import numpy as np
import scipy as sp

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)
print(len(gpus), "Physical GPUs")

# %matplotlib widget
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams.update({'font.size': 16})
mpl.rcParams.update({'font.family': 'serif'})

from utils_data_windows import LOAD_DATA, readSeismicData, getWindowDataDict
from utils_data_windows import get_data_scaler, data_scaler_transform
from utils_data_windows import get_mu_slip_loc_data, get_mu_slip_loc_data_with_index

seed_value = 1234
python_random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [ ]:
verbose = 1

# parameters data dict
params_data = {}
params_data['filenames_train']    = ['./Data/p4677_seismic_data_2MPa.hdf5', './Data/p4677_shear_data_2MPa.hdf5']
params_data['train_data_split']   = 0.6
params_data['valid_data_split']   = 0.4

params_data['in_scaler_mode']     = 'standard'
params_data['out_scaler_mode']    = 'standard'
params_data['sample_freq']        = 1000.0
params_data['unit_time_intvl']    = 0.008
params_data['num_in_time_intvl']  = 32
params_data['num_out_time_intvl'] = 32
params_data['in_time_intvl']      = params_data['unit_time_intvl'] * params_data['num_in_time_intvl']
params_data['out_time_intvl']     = params_data['unit_time_intvl'] * params_data['num_out_time_intvl']
params_data['in_out_time_diff']   = params_data['in_time_intvl']
params_data['time_advance']       = params_data['out_time_intvl']

params_data['batch_size']         = 64
params_data['shuffle_buffer']     = int(1e6)

params_data['tfrecord_window_data'] = './tfrecord_window_data_'+str(params_data['in_time_intvl'])+'_'+str(params_data['out_time_intvl'])+'/'

In [ ]:
# Read Train/Valid data
start_time = time.time()
print('Load Data')

train_data, valid_data, input_scaler, output_scaler = LOAD_DATA(params_data)
params_data['input_scaler']   = input_scaler
params_data['output_scaler']  = output_scaler

print('\nLoad Data Time', time.time()-start_time, 's')

In [ ]:
# Get major slips
params_data['slip_seg_size']      = 10
params_data['slip_range_throld']  = 3.0
params_data['slip_diff_throld']   = -0.4

mu_slip_loc_train, mu_slip_index_train, mu_slip_pos_train = get_mu_slip_loc_data(params_data, train_data['y_data'])
print(mu_slip_loc_train.shape, mu_slip_index_train.shape, mu_slip_pos_train.shape)
print(mu_slip_index_train)

print('')

mu_slip_loc_val, mu_slip_index_val, mu_slip_pos_val = get_mu_slip_loc_data(params_data, valid_data['y_data'])
print(mu_slip_loc_val.shape, mu_slip_index_val.shape, mu_slip_pos_val.shape)
print(mu_slip_index_val)

np.save('./Data/p4677_train_fail_index.npy', mu_slip_pos_train)
np.save('./Data/p4677_valid_fail_index.npy', mu_slip_pos_val)

In [ ]:
filenames_test = ['./Data/p4581_seismic_data_3mpa_stepup.hdf5', './Data/p4581_shear_data_3mpa_stepup.hdf5']

# Read data
start_time = time.time()
print('\nLoad Test Data')
signalData = readSeismicData(filenames_test)
numSignal = signalData['time'].shape[0]

# Input/output scalers
numDataScaler = int(numSignal*0.5)
input_scaler_test = get_data_scaler(signalData['ae'][:numDataScaler], params_data['in_scaler_mode'])
output_scaler_test = get_data_scaler(signalData['mu'][:numDataScaler], params_data['out_scaler_mode'])

# Scaled data
input_signal_test = data_scaler_transform(signalData['ae'], input_scaler_test)
output_signal_test = data_scaler_transform(signalData['mu'], output_scaler_test)

# Test Data
test_data = getWindowDataDict(params_data, input_signal_test, output_signal_test, signalData['time'])

print('\nLoad Data Time', time.time()-start_time, 's')

In [ ]:
# Get major slips
params_data['slip_seg_size']      = 10
params_data['slip_range_throld']  = 2.5
params_data['slip_diff_throld']   = -0.5

mu_slip_loc_test, mu_slip_index_test, mu_slip_pos_test = get_mu_slip_loc_data(params_data, test_data['y_data'])
print(mu_slip_loc_test.shape, mu_slip_index_test.shape, mu_slip_pos_test.shape)
print(mu_slip_index_test)

np.save('./Data/p4581_3mpa_stepup_fail_index.npy', mu_slip_pos_test)